In [64]:
import time
from mysql import connector

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

WAIT_10=10
DATABASE = "guvi"

"""
This method is used to establish connection with MySQL and return the connection
"""
def connect_db():
    try:
        connection = connector.connect(
                host = "localhost",
                user = "root",
                password = "983769"
            )
    except connector.Error as e:
        print(e)
    return connection

"""
This method is used to create the database and the table, if it doesn't exist
"""
def create_db_table(mycursor):
    create_bus_table = """
    CREATE TABLE IF NOT EXISTS bus_routes(
        id INT NOT NULL AUTO_INCREMENT PRIMARY KEY,
        route_name VARCHAR(250),
        route_link VARCHAR(250),
        bus_name VARCHAR(250),
        bus_type VARCHAR(250),
        departing_time TIME,
        duration VARCHAR(50),
        reaching_time TIME,
        star_rating DOUBLE(5,3),
        price DECIMAL(10,3),
        seats_available INT(10)
    );
    """
    # Create Database, if it doesn't exist
    create_db = f"CREATE DATABASE IF NOT EXISTS {DATABASE};"
    use_db = f"use {DATABASE};"
    mycursor.execute(create_db)
    mycursor.execute(use_db)
    # Create table, if it doesn't exist
    mycursor.execute(create_bus_table)

"""
This method is used to insert data inside the table
"""
def insert_row(connection, route_name, route_link, bus_name, bus_type, depart_time, duration, reach_time, star, price, seats):
    try:
        mycursor = connection.cursor()
        sql = "INSERT INTO bus_routes (route_name, route_link, bus_name, bus_type, departing_time\
            , duration, reaching_time, star_rating, price, seats_available) VALUES \
            (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
        record = (route_name, route_link, bus_name, bus_type, depart_time, duration, reach_time, star, price, seats)
        mycursor.execute(sql, record)
        connection.commit()
    except connector.Error as error:
        connection.rollback()
        print("Failed to insert into table {}".format(error))

"""
Open the Government bus page(like APSRTC) and fetch all the route names 
and URLs in the form of Dictionary
"""
def getRouteDict(driver: webdriver, govt_bus_url: str):
    # Open the Government bus page
    driver.get(govt_bus_url)
    driver.maximize_window()
    WebDriverWait(driver, WAIT_5).until(EC.presence_of_element_located((By.CLASS_NAME,'DC_117_pageTabs')))

    # Get the number of pages
    pages = driver.find_elements(By.CLASS_NAME,'DC_117_pageTabs')
    num_pages=len(pages)
    print(f"Pages: {num_pages}")

    route_dict = {}
    for i in range(1, num_pages+1):
        print(f"Start scrape of Page: {i}")
        WebDriverWait(driver, WAIT_5).until(EC.presence_of_element_located((By.CLASS_NAME,'route')))
        # Fetch the route element to get route name and url. (Eg Route: Vijayawada to Hyderabad)
        routes = driver.find_elements(By.CLASS_NAME,'route')

        for j in range(len(routes)):
            route_url=routes[j].get_attribute("href")
            route_title=routes[j].get_attribute("title")
            # print(f"Url: {route_url}, Title: {route_title}")
            # Store the route name and Url in dictionary
            route_dict[tuple((i,j))] = [route_url, route_title]
        # Click Next page to fetch routes from all pages
        if i < num_pages:
            driver.execute_script("arguments[0].click();", pages[i])
    return route_dict

"""
Parse travels details and store in MySQL
"""
def getBusDetails(connection, driver: webdriver, route_dict: dict):
    # skip_till_route = "North Lakhimpur to Dibrugarh"
    # route_flag = False

    for route in route_dict.values():
        # if route[1] != skip_till_route and route_flag:
        #     print(f"Skipping route: {route[1]}")
        #     continue
        # if route_flag:
        #     route_flag = False
        #     continue
        
        # Open the Route URL
        print(f"Route: {route}")
        driver.get(route[0])
        time.sleep(3)

        """
        Based on time, there wouldn't be any buses listed today. 
        Hence, click on Next day to get buses scheduled for tomorrow
        """
        WebDriverWait(driver, WAIT_10).until(EC.element_to_be_clickable((By.CLASS_NAME, "next")))
        driver.execute_script("arguments[0].click();", driver.find_element(By.CLASS_NAME, "next"))
        time.sleep(3)

        # Skip the Routes, where there are no buses present
        if driver.find_elements(By.CLASS_NAME, "oops-page"):
            print(f"No buses found in route:{route[1]}")
            continue
        
        # Click on "View buses" of Government bus to list the Government buses
        WebDriverWait(driver, WAIT_10).until(EC.presence_of_element_located((By.CLASS_NAME, "button")))
        driver.find_element(By.CLASS_NAME, "button").click()

        # Identify the number of buses mentioned in page by parsing only the number
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, WAIT_10).until(EC.presence_of_element_located((By.CLASS_NAME, "busFound")))

        total_bus = driver.find_element(By.CLASS_NAME,'busFound').text
        splitString = total_bus.split(" ")
        total_buses = int( splitString[0] )
        print(f"Total Buses mentioned in page: {total_buses}")

        # Scroll to the page bottom till all the buses are loaded
        last_bus_count=0
        while(True):
            bus_count = len(driver.find_elements(By.CLASS_NAME,'travels'))
            if(bus_count == last_bus_count):
                break
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(3)
            last_bus_count = bus_count

        print(f"Total bus found on {route[1]}: {bus_count}")

        # Fetch the bus details
        bus_route_names     = driver.find_elements(By.CLASS_NAME,'travels')
        bus_type_s          = driver.find_elements(By.CLASS_NAME,'bus-type')
        departure_time_s    = driver.find_elements(By.CLASS_NAME,'dp-time')
        duration_s          = driver.find_elements(By.CLASS_NAME,'dur')
        reaching_time_s     = driver.find_elements(By.CLASS_NAME,'bp-time')
        star_rating_s       = driver.find_elements(By.XPATH,'//span[@class=""]')
        fare_s              = driver.find_elements(By.CLASS_NAME,'fare')
        seats_available_s  = driver.find_elements(By.CLASS_NAME,'seat-left')

        for i in range(bus_count):
            bus_route_name = bus_route_names[i].text
            bus_type = bus_type_s[i].text
            departure_time = departure_time_s[i].text
            duration = duration_s[i].text
            reaching_time = reaching_time_s[i].text
            star_rating = float(star_rating_s[i].text)
            if star_rating > 10:
                star_rating = 10
            fare = fare_s[i].text
            splitString = fare.split(" ")
            fare = float(splitString[1] if len(splitString)>1 else splitString[0])
            seats_available = seats_available_s[i].text
            splitString = seats_available.split(" ")
            seats_available = int(splitString[0])
            # print(route[1], route[0], bus_route_name, bus_type, departure_time\
            #     , duration, reaching_time, star_rating, fare, seats_available)
            insert_row(connection, route[1], route[0], bus_route_name, bus_type, departure_time\
                , duration, reaching_time, star_rating, fare, seats_available)
        print(f"Finished route: {route[1]}")
            

def main():
    driver = webdriver.Chrome()
    connection = connect_db()
    mycursor = connection.cursor()
    create_db_table(mycursor)
    APSRTC_URL = "https://www.redbus.in/online-booking/wbtc-ctc/"
    route_dict = getRouteDict(driver, APSRTC_URL)
    getBusDetails(connection, driver, route_dict)

    mycursor.close()
    connection.close()
    driver.quit()

main()

Pages: 5
Start scrape of Page: 1
Start scrape of Page: 2
Start scrape of Page: 3
Start scrape of Page: 4
Start scrape of Page: 5
Route: ['https://www.redbus.in/bus-tickets/durgapur-to-kolkata', 'Durgapur (West Bengal) to Kolkata']
Total Buses mentioned in page: 114
Total bus found on Durgapur (West Bengal) to Kolkata: 110
Finished route: Durgapur (West Bengal) to Kolkata
Route: ['https://www.redbus.in/bus-tickets/digha-to-barasat-west-bengal', 'Digha to Barasat (West Bengal)']
Total Buses mentioned in page: 35
Total bus found on Digha to Barasat (West Bengal): 29
Finished route: Digha to Barasat (West Bengal)
Route: ['https://www.redbus.in/bus-tickets/barasat-west-bengal-to-digha', 'Barasat (West Bengal) to Digha']
Total Buses mentioned in page: 36
Total bus found on Barasat (West Bengal) to Digha: 30
Finished route: Barasat (West Bengal) to Digha
Route: ['https://www.redbus.in/bus-tickets/suri-to-kolkata', 'Suri to Kolkata']
Total Buses mentioned in page: 14
Total bus found on Suri to